In [1]:
import os
import pandas as pd
current_dir = os.getcwd()
print("initial Working directory",current_dir)
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
print("Final Working Dir",current_dir)

initial Working directory h:\VScode\March Group\March_Team_Project\jupyter_notebooks
You set a new current directory
Final Working Dir h:\VScode\March Group\March_Team_Project


In [2]:
# load pollution_us_2000_2016.csv.zip
pollution = pd.read_csv('source_data/pollution_us_2000_2016.csv.zip', compression='zip')
print(pollution.head())
print(pollution.info())
print(pollution.describe())
# produce summary by city, county, state, counting the number of records, and the number of unique dates add count of unique sites
pollution['Date Local'] = pd.to_datetime(pollution['Date Local'])
pollution['year'] = pollution['Date Local'].dt.year
pollution_summary = pollution.groupby(['City', 'County', 'State', 'year']).agg({'Date Local': ['count', 'nunique']})
pollution_summary.columns = ['Record Count', 'Unique Dates']
print(pollution_summary)

   Unnamed: 0  State Code  County Code  Site Num  \
0           0           4           13      3002   
1           1           4           13      3002   
2           2           4           13      3002   
3           3           4           13      3002   
4           4           4           13      3002   

                                   Address    State    County     City  \
0  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
1  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
2  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
3  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
4  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   

   Date Local          NO2 Units  ...          SO2 Units  SO2 Mean  \
0  2000-01-01  Parts per billion  ...  Parts per billion  3.000000   
1  2000-01-01  Parts per billion  ...  Parts per billion  3.000000   
2  2000-01-01  Parts per billion

In [3]:
# produce a summary by city, county, state, min year with at lwast 350 unique dates and max year with at least 350 unique dates
pollution_summary = pollution_summary.reset_index()
pollution_summary = pollution_summary[pollution_summary['Unique Dates'] >= 350]
pollution_summary = pollution_summary.groupby(['City', 'County', 'State']).agg({'year': ['min', 'max']})
pollution_summary.columns = ['Min Year', 'Max Year']
# difference between min and max year
pollution_summary['Year Difference'] = pollution_summary['Max Year'] - pollution_summary['Min Year']
# sort on Year Difference desc
pollution_summary = pollution_summary.sort_values('Year Difference', ascending=False)

print(pollution_summary)

pollution_data_available = pollution_summary.reset_index()
pollution_data_available

                                              Min Year  Max Year  \
City          County           State                               
Vallejo       Solano           California         2000      2015   
Phoenix       Maricopa         Arizona            2000      2015   
Baton Rouge   East Baton Rouge Louisiana          2000      2015   
Winter Park   Orange           Florida            2000      2015   
Bethel Island Contra Costa     California         2000      2015   
...                                                ...       ...   
Not in a city Rockingham       New Hampshire      2015      2015   
              Linn             Kansas             2002      2002   
              Honolulu         Hawaii             2015      2015   
              Fairfax          Virginia           2006      2006   
York          York             Pennsylvania       2012      2012   

                                              Year Difference  
City          County           State               

,City,County,State,Min Year,Max Year,Year Difference
0,Vallejo,Solano,California,2000,2015,15
1,Phoenix,Maricopa,Arizona,2000,2015,15
2,Baton Rouge,East Baton Rouge,Louisiana,2000,2015,15
3,Winter Park,Orange,Florida,2000,2015,15
4,Bethel Island,Contra Costa,California,2000,2015,15
...,...,...,...,...,...,...
93,Not in a city,Rockingham,New Hampshire,2015,2015,0
94,Not in a city,Linn,Kansas,2002,2002,0
95,Not in a city,Honolulu,Hawaii,2015,2015,0
96,Not in a city,Fairfax,Virginia,2006,2006,0


In [ ]:
# Import ghcnd-stations.txt as fixed width file using the following widths
# ID 1-11, LATITUDE 13-20, LONGITUDE 22-30, ELEVATION 32-37, STATE 39-40, NAME 42-71, GSN FLAG 73-75, HCN/CRN FLAG 77-79, WMO ID 81-85
stations_fixed = pd.read_fwf('source_data/ghcnd-stations.txt', widths=[11, 9, 10, 7, 3, 31, 4, 4, 5], header=None)
stations_fixed.columns = ['StationId', 'Latitude', 'Longitude', 'Elevation', 'StateCode', 'Name', 'GSN Flag', 'HCN/CRN Flag', 'WMO ID']
# set country to first to characters of StationId
stations_fixed['Country'] = stations_fixed['StationId'].str[:2]
stations_fixed = stations_fixed[stations_fixed['Country'] == 'US']
# load ghcnd-states.txt as fixed width file using the following widths
# STATE 1-2, NAME 4-49
states_fixed = pd.read_fwf('source_data/ghcnd-states.txt', widths=[2, 46], header=None)
states_fixed.columns = ['StateCode', 'State']
print("states_fixed", states_fixed.head())
print("stations_fixed", stations_fixed.head())
# match the state name to the state abbreviation
stations_fixed = pd.merge(stations_fixed, states_fixed, on='StateCode')
print("stations_fixed_post_merge",stations_fixed.head())

states_fixed   StateCode           State
0        AB         ALBERTA
1        AK          ALASKA
2        AL         ALABAMA
3        AR        ARKANSAS
4        AS  AMERICAN SAMOA
stations_fixed          StationId  Latitude  Longitude  Elevation StateCode  \
52997  US009052008   43.7333   -96.6333      482.0        SD   
52998  US10RMHS145   40.5268  -105.1113     1569.1        CO   
52999  US10adam001   40.5680   -98.5069      598.0        NE   
53000  US10adam002   40.5093   -98.5493      601.1        NE   
53001  US10adam003   40.4663   -98.6537      615.1        NE   

                                Name GSN Flag HCN/CRN Flag  WMO ID Country  
52997  SIOUX FALLS (ENVIRON. CANADA)      NaN          NaN     NaN      US  
52998                   RMHS 1.6 SSW      NaN          NaN     NaN      US  
52999                  JUNIATA 1.5 S      NaN          NaN     NaN      US  
53000                JUNIATA 6.0 SSW      NaN          NaN     NaN      US  
53001                HOLSTEIN 0.1 

In [12]:
# print pollution_summary columns
print(pollution_summary.columns)
print(pollution_summary.head())
# print states_fixed columns
print(states_fixed.columns)
print(states_fixed.head())
# print stations_fixed columns
print(stations_fixed.columns)
print(stations_fixed.head())

Index(['Min Year', 'Max Year', 'Year Difference'], dtype='object')
                                           Min Year  Max Year  Year Difference
City          County           State                                          
Vallejo       Solano           California      2000      2015               15
Phoenix       Maricopa         Arizona         2000      2015               15
Baton Rouge   East Baton Rouge Louisiana       2000      2015               15
Winter Park   Orange           Florida         2000      2015               15
Bethel Island Contra Costa     California      2000      2015               15
Index(['StateCode', 'State'], dtype='object')
  StateCode           State
0        AB         ALBERTA
1        AK          ALASKA
2        AL         ALABAMA
3        AR        ARKANSAS
4        AS  AMERICAN SAMOA
Index(['StationId', 'Latitude', 'Longitude', 'Elevation', 'StateCode', 'Name',
       'GSN Flag', 'HCN/CRN Flag', 'WMO ID', 'Country', 'State'],
      dtype='object')

In [13]:
# match the state column in pollution_summary to State frp, states_fixed, convert state to upper case for better matching
# Ensure 'State' columns in both dataframes are in uppercase and stripped of leading/trailing spaces for consistent matching
pollution_summary = pollution_summary.reset_index()  # Reset index to access 'State' column directly
pollution_summary['State'] = pollution_summary['State'].str.upper().str.strip()
states_fixed['State'] = states_fixed['State'].str.upper().str.strip()
# merge pollution_summary with states_fixed
pollution_summary_inc_sc = pd.merge(pollution_summary, states_fixed, left_on='State', right_on='State')
print("pollution_summary_post_merge", pollution_summary_inc_sc.head())
# add column to states_fixed with the number of stations in each state
stations_fixed_gb = stations_fixed.groupby(['StateCode', 'State'])['StationId'].count().reset_index()
stations_fixed_gb.columns = ['StateCode', 'State', 'Station Count']
print("stations_fixed_station_count", stations_fixed_gb.head())
# add the station count to the pollution_summary
pollution_summary_stat_count = pd.merge(pollution_summary_inc_sc, stations_fixed_gb, on='StateCode')
print("pollution_summary_ merge2", pollution_summary_stat_count.head())
# add the station count to the states_fixed
states_fixed = pd.merge(states_fixed, stations_fixed_gb, on='State')
print("states_fixed_stations_merge", states_fixed.head())
# sort states_fixed by Station Count desc
states_fixed = states_fixed.sort_values('Station Count', ascending=False)
print("states_fixed_sort", states_fixed)

pollution_summary_stat_count

pollution_summary_post_merge             City         County       State  Min Year  Max Year  \
0        Vallejo         Solano  CALIFORNIA      2000      2015   
1  Bethel Island   Contra Costa  CALIFORNIA      2000      2015   
2         Lompoc  Santa Barbara  CALIFORNIA      2000      2015   
3      San Pablo   Contra Costa  CALIFORNIA      2000      2015   
4        Capitan  Santa Barbara  CALIFORNIA      2000      2015   

   Year Difference StateCode  
0               15        CA  
1               15        CA  
2               15        CA  
3               15        CA  
4               15        CA  
stations_fixed_station_count   StateCode       State  Station Count
0        AK      ALASKA           1049
1        AL     ALABAMA           1151
2        AR    ARKANSAS            961
3        AZ     ARIZONA           1692
4        CA  CALIFORNIA           3166
pollution_summary_ merge2             City         County     State_x  Min Year  Max Year  \
0        Vallejo         S

,City,County,State_x,Min Year,Max Year,Year Difference,StateCode,State_y,Station Count
0,Vallejo,Solano,CALIFORNIA,2000,2015,15,CA,CALIFORNIA,3166
1,Bethel Island,Contra Costa,CALIFORNIA,2000,2015,15,CA,CALIFORNIA,3166
2,Lompoc,Santa Barbara,CALIFORNIA,2000,2015,15,CA,CALIFORNIA,3166
3,San Pablo,Contra Costa,CALIFORNIA,2000,2015,15,CA,CALIFORNIA,3166
4,Capitan,Santa Barbara,CALIFORNIA,2000,2015,15,CA,CALIFORNIA,3166
...,...,...,...,...,...,...,...,...,...
93,Davenport,Scott,IOWA,2014,2015,1,IA,IOWA,1106
94,St. Louis,St. Louis City,MISSOURI,2014,2014,0,MO,MISSOURI,1624
95,Not in a city,Ada,IDAHO,2010,2010,0,ID,IDAHO,846
96,Not in a city,Rockingham,NEW HAMPSHIRE,2015,2015,0,NH,NEW HAMPSHIRE,495


In [ ]:
# generate a list of stationId for those states with 10 or more years of data
states_10_years = pollution_summary_stat_count[pollution_summary_stat_count['Year Difference'] >= 10]


KeyError: 'StationId'

In [17]:
station_list = states_10_years['StationId'].tolist()
station_list

KeyError: 'StationId'